In [ ]:
from ultralytics import YOLO
import cv2
from queue import Queue
import threading

In [ ]:

class_names = ["dog", "cow"] # Nombres de las clases a detectar con yolo

model_general = YOLO('yolo11n.pt')  # Modelo para detectar personas y vehículos

filename = "video_dogs.mp4" # Video a procesar
output_video_filename = "resultado_deteccion.mp4" #Video con la detección y el procesado

intel_dog_data_csv_filename = "dog_intelligence.csv" # Archivo con la información de la inteligencia promedio de los perros en base a la raza
size_dog_data_csv_filename = "AKC_Breed_info.csv" # Archivo con la información del tamaño promedio de los perros en base a la raza



In [ ]:
frame_buffer = Queue()

pause_event = threading.Event()
pause_event.clear()



def frame_to_characteristics():
    while True:
        pause_event.wait()
        frame, track_id, score = frame_buffer.get()
        
        if frame is None: continue
        
        print(f"ID: {track_id}, Score: {score}")
        pause_event.clear()
        

characteristics_thread = threading.Thread(target=frame_to_characteristics)
characteristics_thread.start()

In [ ]:
results = model_general.track(source=filename, stream=True, verbose=False)
frame_count = 0

dog_info = {}

for frame_result in results:
    frame_count += 1
    frame = frame_result.orig_img

    # Imprimir fotograma actual y total de fotogramas
    #print(f'Procesando fotograma {frame_count}')
    
    for box in frame_result.boxes:
        
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        score = box.conf[0].item()
        label = int(box.cls[0])
        track_id = int(box.id[0]) if box.id is not None else -1
        
        #Validación de label  dog = 16  y  Umbral de confianza
        if score>0.6 and label==16 : 
            if (dog_info.get(track_id, [0, "", "", "", ""])[0] < score):
                dog_info[track_id] = [score]
                frame_buffer.put([frame[x1:x2, y1:y2], track_id, score])
                pause_event.set()

        
        if track_id in dog_info:
            cv2.rectangle(frame, (x1,  y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            cv2.putText(frame,  "Dog", (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    
    cv2.imshow('Detección en proceso', frame)

    # Esperar brevemente para permitir visualización, y detener si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

ID: 2, Score: 0.7943414449691772
ID: 2, Score: 0.8188602924346924
ID: 1, Score: 0.7355778813362122
ID: 2, Score: 0.8634712100028992
ID: 4, Score: 0.82431560754776
ID: 35, Score: 0.6606732606887817
ID: 1, Score: 0.7594562768936157
ID: 32, Score: 0.6742120385169983
ID: 35, Score: 0.8192198276519775
ID: 35, Score: 0.8247966766357422
ID: 1, Score: 0.7720242142677307
ID: 44, Score: 0.6857743859291077
ID: 1, Score: 0.789863646030426
ID: 1, Score: 0.801331639289856
ID: 78, Score: 0.6096433401107788
ID: 1, Score: 0.8081077933311462
ID: 4, Score: 0.843058705329895
ID: 1, Score: 0.8828369379043579
ID: 44, Score: 0.6918587684631348
ID: 119, Score: 0.8470113277435303
ID: 44, Score: 0.8757078051567078
ID: 166, Score: 0.7749601602554321
ID: 166, Score: 0.8605273962020874
ID: 166, Score: 0.8893059492111206
ID: 166, Score: 0.9031027555465698
ID: 166, Score: 0.9076392650604248
ID: 166, Score: 0.9140546917915344
ID: 166, Score: 0.9219783544540405
ID: 180, Score: 0.6872431635856628
ID: 166, Score: 0.9227